In [2]:
import librosa
import pyAudioAnalysis as pyaudio
import soundfile as sf
from pydub import AudioSegment
import numpy as np
import pandas as pd
import os

In [2]:
from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import MidTermFeatures

In [3]:
features = []
existing = []
missing = []
prev_features = None

In [4]:
if os.path.exists("../data/features.csv"):
    prev_features = pd.read_csv("../data/features.csv")
    prev_features.dropna(how = 'any')
    prev_features['song'] = prev_features['title'] + " - " + prev_features['artist']
    existing = prev_features['song'].tolist()

In [5]:
for song in os.scandir("."):
    if song.path.endswith(".mp3") and song.is_file():
        # skip ./ in the path to the mp3 file
        file_name = song.path[2:]
        song_name = file_name[:len(file_name) - 4]
        if song_name not in existing:
            # add it to the missing list so we can use pyAudioAnalysis on it
            missing.append(file_name)
            
            # get the tempo of the song
            waveform, samp_rate = librosa.load(file_name)
            tempo, beat_frames = librosa.beat.beat_track(waveform, samp_rate)

            # get the chroma number of the song
            beat_times = librosa.frames_to_time(beat_frames, samp_rate)
            y_harmonic, y_percussive = librosa.effects.hpss(waveform)
            chromagram = librosa.feature.chroma_cqt(y_harmonic, samp_rate)
            beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate = np.median)
            # make beat chroma into a DataFrame and calculate the diff
            chroma_df = pd.DataFrame(beat_chroma)
            diff_values = chroma_df.diff()
            diff_mean = diff_values.mean(axis = 0, skipna = True)
            chroma_num = sum(diff_mean) / len(diff_mean)

            # add song, tempo and chroma number to the features list
            print ([song_name, tempo, chroma_num])
            features.append([song_name, tempo, chroma_num])

/opt/anaconda3/envs/music/lib/python3.8/site-packages/librosa/core/audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


["baby don't cry - exo", 69.83741554054055, -0.01112832059264044]


In [6]:
features_df = pd.DataFrame(features, columns = ['song', 'tempo', 'chroma_number'])
features_df = features_df.sort_values(by = ['song'])

In [7]:
# move all the mp3 files that need to have features extracted into some folder
with open('missing.txt', 'w+') as f:
    for song in missing:
        f.write("%s\n" % song)
# use command line here
'''
mkdir missing
cat missing.txt | while read line
do
    cp $line missing
done
'''
# change the path in directory_feature_extraction

'\nmkdir missing\ncat missing.txt | while read line\ndo\n    cp $line missing\ndone\n'

In [3]:
mid_term_window = 1
mid_term_step = 1
short_term_window = 0.05
short_term_step = 0.05
# use compute_beat = True if we want the extra beat features
pyaudio_feat, files, feat_names = MidTermFeatures.directory_feature_extraction(".", 
                                                                               mid_term_window, 
                                                                               mid_term_step, 
                                                                               short_term_window, 
                                                                               short_term_step,
                                                                               False)

Analyzing file 1 of 1: trial/traveler of the magic borders - takanashi yasuharu.mp3
Feature extraction complexity ratio: 43.9 x realtime


In [25]:
# neural network features
features_df.reset_index(drop = True, inplace = True)
if len(features_df) == 1:
    nn_features_df = pd.DataFrame([pyaudio_feat], columns = feat_names)
else:
    nn_features_df = pd.DataFrame(pyaudio_feat, columns = feat_names)
nn_features_df['song'] = features_df['song']

In [26]:
zcr_ind = feat_names.index('zcr_mean')
ee_ind = feat_names.index('energy_entropy_mean')
spc_ind = feat_names.index('spectral_centroid_mean')

features_df['zero_crossing_rate'] = nn_features_df.iloc[:, zcr_ind]
features_df['energy_entropy'] = nn_features_df.iloc[:, ee_ind]
features_df['spectral_centroid'] = nn_features_df.iloc[:, spc_ind]

In [27]:
features_df

,song,tempo,chroma_number,zero_crossing_rate,energy_entropy,spectral_centroid
0,baby don't cry - exo,69.837416,-0.011128,0.053817,3.185847,0.136686


In [62]:
moods = pd.read_csv("../data/mood_data.csv")
moods

,title,artist,primary,secondary
0,who says,selena gomez,2,3
1,don't stop,5 seconds of summer,3,2
2,do you hear what i hear,carrie underwood,4,2
3,"alliance force, assemble",takanashi yasuharu,1,6
4,ni pa bu pa shi qu wo,liu zeng tong,2,4
...,...,...,...,...
195,nine four two zero,mai xiao er,2,4
196,zui mei hun li,bai xiao bai,2,4
197,jiu shi xi huan ni,li meng yin,2,6
198,xiao xing xing,wang su long,2,4


In [63]:
# merging is incorrect, does not work
moods['song'] = moods['title'] + " - " + moods['artist']
moods = moods.sort_values(by = ['song'])
moods.reset_index(drop = True, inplace = True)

new_features = features_df.merge(moods, on = "song", how = "left")
new_features = new_features.drop(columns = ['song'])
new_features

,tempo,chroma_number,zero_crossing_rate,energy_entropy,spectral_centroid,title,artist,primary,secondary
0,69.837416,-0.011128,0.053817,3.185847,0.136686,"baby, don't cry",exo,2,4


In [15]:
if not existing:
    new_features = new_features[['title', 'artist', 'tempo', 'chroma_number', 
                                 'zero_crossing_rate', 'energy_entropy', 
                                 'spectral_centroid', 'primary', 'secondary']]
    new_features.to_csv("../data/features.csv", header = True, index = False)
    
    nn_features_df.to_csv("../data/nn_features.csv", header = True, index = False)
    
    feat_names_df = pd.DataFrame(feat_names)
    feat_names_df.to_csv("../data/nn_feature_names.csv", header = True, index = False)
else:
    prev_features = prev_features.drop(columns = ['song'])
    all_features = pd.concat([prev_features, new_features], ignore_index = True)
    all_features = all_features[['title', 'artist', 'tempo', 'chroma_number', 
                                 'zero_crossing_rate', 'energy_entropy', 
                                 'spectral_centroid', 'primary', 'secondary']]
    all_features.to_csv("../data/features.csv", header = True, index = False)
    
    prev_nn = pd.read_csv("../data/nn_features.csv")
    all_nn = pd.concat([prev_nn, nn_features_df], ignore_index = True)
    all_nn.to_csv("../data/nn_features.csv", header = True, index = False)

In [ ]:
# make sure to delete the directory missing and the text file missing.txt
'''
rm -r missing
rm missing.txt
'''

In [18]:
# extract additional neural network features
# ! check ffmpeg at some point !
existing_raw = []
new_raw_dict = {}

In [19]:
# check existing raw features
if os.path.exists("../data/raw_features.csv"):
    prev_raw = pd.read_csv("../data/raw_features.csv")
    prev_raw.dropna(how = 'any')
    prev_raw['song'] = prev_raw['title'] + " - " + prev_raw['artist']
    existing_raw = prev_raw['song'].tolist()

In [20]:
for song in os.scandir("."):
    if song.path.endswith(".mp3") and song.is_file():
        # skip ./ in the path to the mp3 file
        file_name = song.path[2:]
        song_name = file_name[:len(file_name) - 4]
        if song_name not in existing_raw:
            sound = AudioSegment.from_mp3(file_name)
            sound = sound.set_channels(1)
            sound.export("temp.wav", format = "wav")
            # does not work for mp3s that are too short or have an extremely low sample rate
            for block in sf.blocks("temp.wav", blocksize = 25000, start = 750000):
                new_raw_dict[song_name] = block
                break
if os.path.exists("temp.wav"):
    os.remove("temp.wav")
else:
    print ("No temporary wav file found, extraction failed.")

In [21]:
new_raw = pd.DataFrame(new_raw_dict)
new_raw = new_raw.transpose()
new_raw.index.name = 'song'
new_raw.reset_index(inplace = True)
new_raw = new_raw.sort_values(by = ['song'])
new_raw.reset_index(drop = True, inplace = True)

In [22]:
new_raw

,song,0,1,2,3,4,5,6,7,8,...,24990,24991,24992,24993,24994,24995,24996,24997,24998,24999
0,a new adventure - takanashi yasuharu,0.068237,0.109283,0.099518,0.098816,0.103577,0.094940,0.108521,0.135986,0.176697,...,-0.104279,0.093658,0.068115,0.078491,0.020477,0.050995,0.087891,0.024353,0.088715,0.054199
1,a oh - super junior,0.220276,0.224091,0.213287,0.167145,0.117676,0.112976,0.138336,0.149017,0.132080,...,0.273712,0.182556,0.125397,0.102173,0.074829,0.045105,0.044586,0.056671,0.058136,0.070831
2,a thousand years - christina perri,-0.026520,-0.020630,-0.013947,-0.007935,-0.004211,-0.003632,-0.005859,-0.009949,-0.015137,...,0.020752,0.026337,0.031799,0.037384,0.042328,0.046112,0.048615,0.049683,0.050385,0.051056
3,adore u - seventeen,0.183411,0.180908,0.180389,0.178650,0.172821,0.166351,0.161469,0.157501,0.155121,...,-0.427673,-0.445374,-0.468323,-0.489227,-0.505035,-0.515198,-0.517456,-0.513184,-0.503998,-0.488831
4,after rain - aimer,-0.018036,-0.024689,-0.028870,-0.031891,-0.031586,-0.030762,-0.041412,-0.061859,-0.079803,...,-0.230804,-0.256378,-0.274872,-0.289337,-0.296051,-0.293579,-0.283508,-0.265594,-0.245941,-0.234711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,you he bu ke - xu song,-0.180023,-0.173584,-0.167053,-0.160553,-0.155060,-0.150421,-0.145905,-0.141602,-0.137787,...,0.454865,0.442719,0.449585,0.485291,0.498291,0.504639,0.518982,0.534424,0.551941,0.566895
196,youth - troye sivan,0.185028,0.163757,0.153687,0.145569,0.128662,0.109772,0.106537,0.120972,0.129669,...,-0.110107,-0.143768,-0.157898,-0.149994,-0.134827,-0.126831,-0.130737,-0.135986,-0.126129,-0.109039
197,"zhe shi ai - henry, donghae",0.024811,0.025848,0.026703,0.027344,0.028015,0.028503,0.028900,0.029175,0.029419,...,0.037872,0.035004,0.031738,0.028076,0.024048,0.019806,0.015350,0.010834,0.006409,0.002075
198,zui mei hun li - bai xiao bai,-0.030304,-0.023560,-0.024353,-0.030243,-0.023590,-0.013855,-0.017548,-0.024994,-0.024170,...,-0.287048,-0.339417,-0.331390,-0.239288,-0.147339,-0.111084,-0.104614,-0.171692,-0.277710,-0.279907


In [23]:
if not existing_raw:
    new_raw.to_csv("../data/raw_features.csv", header = True, index = False)
else:
    all_raw = pd.concat([prev_raw, new_raw], ignore_index = True)
    all_raw.to_csv("../data/raw_features.csv", header = True, index = False)

In [3]:
# feature engineering
feat_names_df = pd.read_csv("../data/nn_feature_names.csv")
current_features = pd.read_csv("../data/features.csv")
feats_to_extract = pd.read_csv("../data/nn_features.csv")
engineered_features = current_features[['title', 'artist', 'tempo', 'chroma_number']]

# check that the ordering of songs is the same
engineered_features['song'] = feats_to_extract['song']
engineered_features['song_check'] = engineered_features['title'] + ' - ' + engineered_features['artist']
if engineered_features['song'].equals(engineered_features['song_check']):
    print ("Song ordering matches.")
    engineered_features = engineered_features.drop(columns = ['song', 'song_check'])
else:
    print ("Song ordering DOES NOT match.")

# add or remove additional features
feat_names_list = feat_names_df['0'].to_list()
# selected 28 total features based on distribution boxplots by primary mood
selected_feats = ['zcr_mean', 'zcr_std', 'energy_mean', 'energy_entropy_mean', 'spectral_centroid_mean', 
                  'spectral_spread_mean', 'spectral_entropy_mean', 'mfcc_2_mean', 'mfcc_5_mean', 'mfcc_6_mean',
                  'spectral_centroid_std', 'spectral_entropy_std', 'spectral_spread_std', 'chroma_7_std',
                  'delta chroma_2_std', 'delta chroma_3_std', 'delta chroma_9_std', 'delta chroma_std_std',
                  'delta energy_std', 'delta mfcc_1_std', 'delta mfcc_3_std', 'delta mfcc_13_std',
                  'delta spectral_centroid_std', 'delta spectral_entropy_std', 'delta spectral_flux_std',
                  'delta spectral_spread_std']
for feat in selected_feats:
    engineered_features[feat] = feats_to_extract.iloc[:, feat_names_list.index(feat)]

# add mood labels to the final features dataframe
engineered_features['primary'] = current_features['primary']
engineered_features['secondary'] = current_features['secondary']

# save the engineered features
engineered_features.to_csv("../data/engineered_features.csv", header = True, index = False)
engineered_features

Song ordering matches.


<ipython-input-3-375b915b913a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engineered_features['song'] = feats_to_extract['song']
<ipython-input-3-375b915b913a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  engineered_features['song_check'] = engineered_features['title'] + ' - ' + engineered_features['artist']


,title,artist,tempo,chroma_number,zcr_mean,zcr_std,energy_mean,energy_entropy_mean,spectral_centroid_mean,spectral_spread_mean,...,delta energy_std,delta mfcc_1_std,delta mfcc_3_std,delta mfcc_13_std,delta spectral_centroid_std,delta spectral_entropy_std,delta spectral_flux_std,delta spectral_spread_std,primary,secondary
0,a new adventure,takanashi yasuharu,117.453835,-0.010635,0.082014,0.031156,0.061664,3.164051,0.186062,0.201923,...,0.031983,0.619485,0.254529,0.189750,0.039689,0.304516,0.003464,0.026444,1,6
1,a oh,super junior,129.199219,-0.005782,0.078039,0.028662,0.087948,3.213386,0.179459,0.196234,...,0.031600,0.785050,0.345785,0.225282,0.035337,0.333457,0.002662,0.021355,3,2
2,a thousand years,christina perri,92.285156,-0.016321,0.044617,0.018092,0.052477,3.217903,0.122203,0.170703,...,0.016750,0.521221,0.241593,0.196299,0.028887,0.155287,0.002794,0.030025,6,5
3,adore u,seventeen,103.359375,-0.007745,0.058125,0.027802,0.113696,3.142738,0.145660,0.170424,...,0.064444,1.280268,0.469696,0.248865,0.035615,0.324714,0.007247,0.028047,3,2
4,after rain,aimer,117.453835,-0.010324,0.068620,0.026402,0.083934,3.211794,0.147982,0.169066,...,0.064776,1.005204,0.362243,0.220264,0.033298,0.300292,0.005283,0.025979,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,you he bu ke,xu song,99.384014,0.015339,0.056268,0.042850,0.073693,3.166332,0.156298,0.198088,...,0.041610,1.132257,0.402372,0.242564,0.071073,0.410712,0.009546,0.050956,2,4
196,youth,troye sivan,123.046875,-0.002242,0.056377,0.035249,0.081590,3.192966,0.140693,0.169824,...,0.036208,0.903736,0.388618,0.235086,0.039178,0.372678,0.005693,0.027476,5,6
197,zhe shi ai,"henry, donghae",151.999081,-0.020014,0.019520,0.004853,0.010187,3.202357,0.075014,0.148539,...,0.005241,1.118669,0.309914,0.191399,0.036687,0.037481,0.012192,0.058416,4,6
198,zui mei hun li,bai xiao bai,112.347147,0.022650,0.051512,0.039506,0.056785,3.168251,0.134493,0.169568,...,0.025834,1.111881,0.477303,0.251263,0.042643,0.410848,0.009373,0.031184,2,4


In [31]:
feat_names_list

['zcr_mean',
 'energy_mean',
 'energy_entropy_mean',
 'spectral_centroid_mean',
 'spectral_spread_mean',
 'spectral_entropy_mean',
 'spectral_flux_mean',
 'spectral_rolloff_mean',
 'mfcc_1_mean',
 'mfcc_2_mean',
 'mfcc_3_mean',
 'mfcc_4_mean',
 'mfcc_5_mean',
 'mfcc_6_mean',
 'mfcc_7_mean',
 'mfcc_8_mean',
 'mfcc_9_mean',
 'mfcc_10_mean',
 'mfcc_11_mean',
 'mfcc_12_mean',
 'mfcc_13_mean',
 'chroma_1_mean',
 'chroma_2_mean',
 'chroma_3_mean',
 'chroma_4_mean',
 'chroma_5_mean',
 'chroma_6_mean',
 'chroma_7_mean',
 'chroma_8_mean',
 'chroma_9_mean',
 'chroma_10_mean',
 'chroma_11_mean',
 'chroma_12_mean',
 'chroma_std_mean',
 'delta zcr_mean',
 'delta energy_mean',
 'delta energy_entropy_mean',
 'delta spectral_centroid_mean',
 'delta spectral_spread_mean',
 'delta spectral_entropy_mean',
 'delta spectral_flux_mean',
 'delta spectral_rolloff_mean',
 'delta mfcc_1_mean',
 'delta mfcc_2_mean',
 'delta mfcc_3_mean',
 'delta mfcc_4_mean',
 'delta mfcc_5_mean',
 'delta mfcc_6_mean',
 'delta m